Import Libraries

In [18]:
from pyvis.network import Network
import random

Define initial parameters

In [19]:
# define time steps
timeSteps = 100

# define fields
fieldColors = {"CS": "blue", 
                "Math": "green", 
                "Physics": "red"}
fields = list(fieldColors.keys())


Create Model

In [20]:
net = Network()
scholarField = random.choice(fields)
net.add_node(0, label=scholarField, color=fieldColors[scholarField])

# go through time steps, add new scholar and paper at each step
for i in range(1, timeSteps):

    # add new paper

    # add new scholar
    scholarField = random.choice(fields)
    net.add_node(i, label=scholarField, color=fieldColors[scholarField])

Display Network

In [21]:
net.show('testModel.html')